# Importando bibliotecas necessárias

In [26]:
import os
import openai
from dotenv import load_dotenv, find_dotenv  # Para carregar variáveis de ambiente do arquivo .env
import time

# Carregando as variáveis de ambiente a partir do arquivo .env

In [2]:
# Carregando as variáveis de ambiente a partir do arquivo .env
_ = load_dotenv(find_dotenv())

# Recuperando a chave da API OpenAI armazenada no arquivo .env
openai_api_key = os.getenv("OPENAI_API_KEY")

# Configurando a chave da API para autenticar solicitações à OpenAI

In [3]:
openai.api_key = openai_api_key

# Definindo o cliente para interagir com a API OpenAI

In [4]:
client = openai.Client()

# Criando uma mensagem de exemplo para a API

In [5]:
# Aqui, simulamos uma pergunta do usuário sobre uma maçã
mensagens = [{'role' : 'user', 'content' : 'O que é uma maça em 5 palavras?'}]

# Fazendo uma solicitação para o modelo 'gpt-3.5-turbo-0125'

In [6]:
# Fazendo uma solicitação para o modelo 'gpt-3.5-turbo-0125'
# max_tokens: o máximo de tokens que a resposta pode ter
# temperature: controla a criatividade (0 = mais determinístico, menos criativo)

resposta = client.chat.completions.create(
    messages=mensagens,
    model='gpt-3.5-turbo-0125',
    max_tokens=1000,
    temperature=0
)

# Exibindo a resposta gerada

In [7]:
print(resposta.choices[0].message.content)

Fruta redonda e saborosa.


# Adicionando a resposta do assistente à lista de mensagens

In [8]:
# A função append insere a resposta do modelo como se fosse o "assistant" respondendo
mensagens.append({'role': 'assistant', 'content': resposta.choices[0].message.content})

# Adicionando uma nova mensagem do usuário à lista de mensagens

In [9]:
# Essa mensagem simula o usuário perguntando "E qual é a sua cor?"
# A mensagem é adicionada com o papel ('role') de 'user', mantendo o fluxo da conversa.
mensagens.append({'role': 'user', 'content': 'E qual é a sua cor?'})

# Exibindo a respostas geradas

In [10]:
mensagens

[{'role': 'user', 'content': 'O que é uma maça em 5 palavras?'},
 {'role': 'assistant', 'content': 'Fruta redonda e saborosa.'},
 {'role': 'user', 'content': 'E qual é a sua cor?'}]

# Fazendo uma nova solicitação para o modelo 'gpt-3.5-turbo-0125' com base nas mensagens acumuladas

In [11]:
# - messages: a lista de mensagens (perguntas e respostas) que define o contexto da conversa
# - model: o modelo de linguagem a ser usado (neste caso, 'gpt-3.5-turbo-0125')
# - max_tokens: o número máximo de tokens que a resposta pode ter
# - temperature: define a aleatoriedade da resposta (0 significa respostas mais previsíveis e determinísticas)

resposta = client.chat.completions.create(
    messages=mensagens,
    model='gpt-3.5-turbo-0125',
    max_tokens=1000,
    temperature=0
)

# Exibindo o conteúdo da resposta gerada pelo assistente

In [12]:
# A resposta está localizada na primeira escolha ('choices[0]'), dentro da chave 'message.content'
print(resposta.choices[0].message.content)

Vermelha, verde ou amarela.


# Gerando uma função para facilitar o processo

A seguir, vamos definir uma função chamada `geracao_texto` que facilita a interação com a API do ChatGPT.
Essa função permite enviar mensagens, receber respostas e manter o histórico da conversa de maneira automática.

### Parâmetros da Função:
- **mensagens**: Lista contendo o histórico de mensagens anteriores.
- **model**: O modelo de linguagem a ser utilizado (padrão: `gpt-3.5-turbo-0125`).
- **max_tokens**: Define o número máximo de tokens permitidos na resposta (padrão: 1000).
- **temperature**: Controla o nível de aleatoriedade da resposta. Um valor de 0 produz respostas mais determinísticas.

A função retorna o histórico atualizado das mensagens com a nova resposta incluída.


In [13]:
def geracao_texto(mensagens, model='gpt-3.5-turbo-0125', max_tokens=1000, temperature=0):
    """
    Função para gerar texto usando a API do ChatGPT com base nas mensagens passadas.
    
    Parâmetros:
    - mensagens: lista de dicionários que contém as mensagens anteriores (conversa).
    - model: o modelo de linguagem a ser usado (padrão: 'gpt-3.5-turbo-0125').
    - max_tokens: o número máximo de tokens que a resposta pode ter (padrão: 1000).
    - temperature: define a aleatoriedade da resposta (padrão: 0, mais determinístico).
    """

    # Fazendo uma solicitação à API do ChatGPT com base nas mensagens e parâmetros fornecidos.
    resposta = client.chat.completions.create(
        messages=mensagens,
        model=model,  # Modelo de linguagem usado (exemplo: 'gpt-3.5-turbo-0125')
        max_tokens=max_tokens,  # Limita o tamanho da resposta
        temperature=temperature  # Controla a criatividade da resposta (0 = mais previsível)
    )

    # Exibindo a resposta gerada pelo assistente
    print(resposta.choices[0].message.content)

    # Adiciona a resposta gerada pelo assistente à lista de mensagens.
    # A linha comentada (mensagens.append...) exigiria que você manualmente dissesse se o conteúdo vem do user ou assistant.
    # A linha ativa abaixo usa 'model_dump()' para automaticamente formatar e estruturar a resposta de acordo com o formato correto, excluindo valores None.
    mensagens.append(resposta.choices[0].message.model_dump(exclude_none=True))  # Inclui a resposta gerada no histórico de mensagens.

    # Retorna a lista atualizada de mensagens com a nova resposta do assistente incluída.
    return mensagens


# Inicializando a lista de mensagens com uma pergunta do usuário.

In [14]:
# A primeira mensagem contém o papel (role) de 'user' e a pergunta "O que é uma maçã em 5 palavras?"
mensagens = [{'role' : 'user', 'content' : 'O que é uma maça em 5 palavras?'}]

# Chamando a função 'geracao_texto' para enviar a mensagem à API e obter a resposta.

In [15]:
# A função retorna o histórico atualizado de mensagens com a resposta do assistente incluída.
mensagens = geracao_texto(mensagens)

Fruta redonda e saborosa.


# Adicionando uma nova mensagem do usuário à lista de mensagens.

In [16]:
# O usuário faz uma nova pergunta: "E qual é a sua cor?"
mensagens.append({'role': 'user', 'content': 'E qual é a sua cor?'})

# Chamando novamente a função 'geracao_texto' com a lista atualizada de mensagens.

In [17]:
# A função envia a nova pergunta para a API e retorna o histórico de mensagens atualizado com a resposta do assistente.
mensagens = geracao_texto(mensagens)

Vermelha, verde ou amarela.


# Explorando as diferenças de acordo com a quantidade de tokens

In [18]:
# Definindo 'max_tokens=20', limitando a resposta da API a apenas 5 tokens.
# Isso ainda resulta em uma resposta curta, mas mais completa que com 2 tokens.
mensagens = [{'role' : 'user', 'content' : 'O que é uma maça?'}]
mensagens = geracao_texto(mensagens, max_tokens=20)

# Repetindo a pergunta "O que é uma maçã?" 
# Definindo 'max_tokens=30', permitindo uma resposta significativamente maior e mais detalhada.
mensagens = [{'role' : 'user', 'content' : 'O que é uma maça?'}]
mensagens = geracao_texto(mensagens, max_tokens=30)

# Fazendo uma nova pergunta: "O que é uma maçã?"
# Definindo 'max_tokens=100' para permitir uma resposta mais longa.
# Também ajustando 'temperature=0', garantindo que a resposta seja mais determinística (menos aleatória e mais precisa).
mensagens = [{'role' : 'user', 'content' : 'O que é uma maça?'}]
mensagens = geracao_texto(mensagens, max_tokens=100, temperature=0)

Uma maçã é uma fruta redonda, geralmente de cor vermelha,
Uma maçã é uma fruta redonda, geralmente de cor vermelha, verde ou amarela, com polpa bran
Uma maçã é uma fruta redonda, geralmente de cor vermelha, verde ou amarela, com polpa branca e suculenta. É uma das frutas mais populares e consumidas em todo o mundo, sendo conhecida por seu sabor doce e refrescante. A maçã é rica em fibras, vitaminas e minerais, sendo considerada uma fruta muito saudável. Ela pode ser consum


# Explorando as diferenças de acordo com a 'temperature'

In [19]:
# Fazendo uma nova pergunta: "O que é uma maçã?"
# Definindo 'max_tokens=30' para permitir uma resposta mais longa (apesar da pergunta solicitar no máximo 10 palavras).
# Também ajustando 'temperature=0', garantindo uma resposta mais determinística e precisa (menos criativa e mais direta).
mensagens = [{'role' : 'user', 'content' : 'O que é uma maça em no maximo 10 palavras?'}]
mensagens = geracao_texto(mensagens, max_tokens=30, temperature=0)

# Repetindo a pergunta "O que é uma maçã?" ,
# mas agora com 'temperature=1', o que aumenta a criatividade do modelo,
# permitindo respostas mais variadas e menos previsíveis.
mensagens = [{'role' : 'user', 'content' : 'O que é uma maça em no maximo 10 palavras?'}]
mensagens = geracao_texto(mensagens, max_tokens=30, temperature=1)

# Repetindo a pergunta "O que é uma maçã?" 
# Definindo 'temperature=2', o que resulta em uma resposta altamente criativa e potencialmente imprevisível,
# aumentando a variedade e a originalidade na geração do texto.
mensagens = [{'role' : 'user', 'content' : 'O que é uma maça em no maximo 10 palavras em portugues?'}]
mensagens = geracao_texto(mensagens, max_tokens=30, temperature=2)

Fruta redonda, suculenta e crocante, geralmente de cor vermelha.
Fruta redonda, suculenta e doce, de casca verde ou vermelha.
Uma fruta redonda com polpa suculenta e sabor doce.


# Adicionando stream na função de geração de texto

### O que o parâmetro `stream` faz?

O parâmetro `stream`, quando definido como `True`, permite que as respostas da API do ChatGPT sejam transmitidas em tempo real, pedaço por pedaço (chunks), à medida que são geradas. Isso é especialmente útil em casos onde a resposta é muito longa ou quando se deseja atualizar uma interface com novas partes do texto assim que elas ficam disponíveis.

#### Benefícios do `stream`:
- **Respostas longas**: Para evitar esperar pela resposta completa, o `stream` permite que partes do conteúdo sejam recebidas e exibidas imediatamente.
- **Melhor experiência do usuário**: Em uma interface interativa, a exibição


In [38]:
def geracao_texto_e_print_simultaneo(mensagens, model='gpt-3.5-turbo-0125', max_tokens=1000, temperature=0, stream=True):
    """
    Função para gerar texto usando a API do ChatGPT com base nas mensagens passadas.
    
    Parâmetros:
    - mensagens: lista de dicionários que contém as mensagens anteriores (conversa).
    - model: o modelo de linguagem a ser usado (padrão: 'gpt-3.5-turbo-0125').
    - max_tokens: o número máximo de tokens que a resposta pode ter (padrão: 1000).
    - temperature: define a aleatoriedade da resposta (padrão: 0, mais determinístico).
    - stream: se True, a resposta é transmitida em tempo real, recebendo pedaços do conteúdo conforme ele é gerado.
    """

    resposta_completa = ""

    # Fazendo uma solicitação à API do ChatGPT com base nas mensagens e parâmetros fornecidos.
    resposta = client.chat.completions.create(
        messages=mensagens,
        model=model,  # Modelo de linguagem usado (exemplo: 'gpt-3.5-turbo-0125')
        max_tokens=max_tokens,  # Limita o tamanho da resposta
        temperature=temperature,  # Controla a criatividade da resposta (0 = mais previsível)
        stream=stream
    )

    for stream_resposta in resposta:
        texto = stream_resposta.choices[0].delta.content

        if texto:
            resposta_completa += texto
            time.sleep(0.1)
            print(texto, end="")

    return mensagens, resposta_completa

# Chamando a função 'geracao_texto_e_print_simultaneo' para perguntar e ter a resposta simultaneamente

In [39]:
mensagens = [{'role' : 'user', 'content' : 'O que é uma maça?'}]

mensagens, resposta_completa = geracao_texto_e_print_simultaneo(mensagens, max_tokens=30)

Uma maçã é uma fruta redonda, geralmente de cor vermelha, verde ou amarela, com polpa bran

In [40]:
resposta_completa

'Uma maçã é uma fruta redonda, geralmente de cor vermelha, verde ou amarela, com polpa bran'